#### Libraries

In [ ]:
!pip install tf2onnx
!pip install onnxruntime
import tensorflow as tf
import tf2onnx
import onnx
import onnxruntime as rt
import numpy as np
import cv2
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from datetime import datetime
from onnxruntime.quantization import quantize_dynamic, QuantType
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 39.2 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00
Mounted at /content/drive


#### Process Dataset

In [ ]:
def read_image(path):
    #Read image from path using OpenCV.
    img = cv2.imread(path)
    #Resize image to 256x256x3.
    img = cv2.resize(img,(256,256))
    #Normalize image.
    img = img/255.0
    return img

def read_target(path):
    #Read image from path as greyscale using OpenCV.
    msk = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    #Resize target to 256x256.
    msk = cv2.resize(msk,(256,256))
    #Define target at floating point.
    msk = msk/1.0
    #Add one dimension to image array.
    msk = np.expand_dims(msk,axis=-1)
    return msk

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def SSIMLoss(y_true, y_pred):
    SSIMLoss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))
    return SSIMLoss

#### Convert TF keras model to ONNX format.

In [ ]:
#Read Keras model from path.
model = tf.keras.models.load_model("/PATH/FlareNet_xxx.h5", custom_objects={'SSIMLoss':SSIMLoss})
#Define input tensor type and name.
format = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input_image"),)
#Convert Keras model to ONNX. Optset depends on the Jetpack version, in this case it has to be 12 because 13 is not supported.
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature = format, opset=12)
#Define path to save ONNX model.
onnx_path = "/PATH/FlareNet_xxx.onnx"
#Save ONNX model.
onnx.save(onnx_model, onnx_path)